# Data Integration on MediaEval Dev Dataset

- Data cleaning on development set
- We will look at the images that had been unzipped, process the images and save it into a dataframe
- There are 3 files that we are working on which will be combined later
    1. tweets.txt - The data contains the tweet text
    2. users_features.csv - The data contain social information such as #friends, #followers, etc. 
    3. tweet_features.csv - The data contain tweet features such as number of positive emoticon, order pron, etc.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import sys, os, requests, glob
from PIL import Image

In [2]:
df_path = "./MediaEval/mediaeval2015/devset"

### Checking Each Image

1. Open each image, if there is an error that append the row to remove_list
2. If the file extension of the image is not jpg, then save the file to jpg extension and delete the original file
3. Drop all images that is in the remove_list

We will drop rows that is in the remove_list, this allows us to work only with valid images

The invalid images will arise during the downloading of the images from sites such as Instagram

In [3]:
images_path = os.path.join(df_path, 'Medieval2015_DevSet_Images')

valid_list = []
path_list = []
event_list = []

for root, dirs, files in os.walk(images_path):
    if len(dirs) == 0:
        for file in files:
            file_split = file.split('.')
            event = file.split('_')[0]
            path = os.path.join(root, file)
            img = Image.open(os.path.join(root, file))
            if file_split[-1] != 'jpg':
                try:
                    file_save = os.path.join(root, file_split[0] + '.jpg')
                    img.save(file_save)
                    os.remove(path)
                except:
                    os.remove(path)
                    os.remove(file_save)
            valid_list.append(file_split[0])
            path_list.append(path)
            event_list.append(event)

df_images = pd.DataFrame({'imageId': valid_list, 'imagePath': path_list, 'event': event_list})

In [4]:
# Convert the image id of sandyA and sandyB to sandy 
df_images['event'] = df_images['event'].apply(lambda x : 'sandy' if ((x == 'sandyA') | (x == 'sandyB')) else x)

### Save to Pickle

In [5]:
df_images.to_pickle('dev_images.pkl')

## Data Cleaning on Devset Tweets

- Objective is to extract the tweets
- There are duplicates tweets in the data, so we will drop the duplicates by its tweetId. tweetId should be unique

In [6]:
df_raw_tweets_path = os.path.join(df_path, 'tweets.txt')
df_raw_tweets = pd.read_csv(df_raw_tweets_path, sep = "\t")
df_raw_tweets.head()

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake


In [7]:
df_raw_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 7 columns):
tweetId       14277 non-null int64
tweetText     14277 non-null object
userId        14277 non-null int64
imageId(s)    14277 non-null object
username      14277 non-null object
timestamp     14277 non-null object
label         14277 non-null object
dtypes: int64(2), object(5)
memory usage: 780.9+ KB


### Duplicates Tweets

- Drop duplicates from tweetId which should be unique

In [8]:
# Drop duplicate tweets
df_raw_tweets.drop_duplicates('tweetId', inplace = True)

### Save to Pickle

In [9]:
df_raw_tweets.to_pickle('dev_raw_tweets.pkl')

## Data Cleaning on Devset User Features

- Objective is to extract some user features from the data such as #friends, #followers etc.
- Similar to previous step, we will drop duplicates from tweetId

In [10]:
df_user_path = os.path.join(df_path, 'DatasetFeatures/user_features.csv') 
df_user = pd.read_csv(df_user_path)
df_user.head()

,tweet_id,num_friends,num_followers,folfriend_ratio,times_listed,has_url,is_verified,num_tweets
0,263046056240115712,283,1651,5.833922,64,False,False,43811
1,262995061304852481,2189,95637,43.689810,737,True,False,54293
2,262979898002534400,1972,1701,0.862576,13,False,False,34414
3,262996108400271360,267,235,0.880150,0,False,False,17837
4,263018881839411200,193,181,0.937824,0,False,False,25754


In [11]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14483 entries, 0 to 14482
Data columns (total 8 columns):
tweet_id            14483 non-null int64
 num_friends        14483 non-null int64
 num_followers      14483 non-null int64
 folfriend_ratio    14483 non-null float64
 times_listed       14483 non-null int64
 has_url            14483 non-null bool
 is_verified        14483 non-null bool
 num_tweets         14483 non-null int64
dtypes: bool(2), float64(1), int64(5)
memory usage: 707.3 KB


### Columns name cleaning

- Remove leading whitespace in the column name

In [12]:
df_user.columns = df_user.columns.str.strip()

### Duplicates Tweets

- Drop duplicates from tweetId which should be unique

In [13]:
df_user.drop_duplicates('tweet_id', inplace = True)

### Save to Pickel

In [14]:
df_user.to_pickle('dev_users.pkl')

## Data Cleaning on Tweets Features

- Objective is to extract tweet features from tweets such as num_words, contains quest_marks, etc.
- We will handle some of the null values 

In [15]:
# Can skip
df_tweets_path = os.path.join(df_path, 'DatasetFeatures/tweet_features.csv') 
df_tweets_features = pd.read_csv(df_tweets_path)
df_tweets_features.head()

,tweet_id,num_words,text_length,contains_questmark,num_questmark,contains_exclammark,num_exclammark,contains_happyemo,contains_sademo,contains_firstorderpron,contains_secondorderpron,contains_thirdorderpron,num_uppercasechars,num_possentiwords,num_negsentiwords,num_mentions,num_hashtags,num_URLs,num_retweets
0,263046056240115712,22,134,True,1,False,0,False,False,False,False,True,3,0.0,0.0,0,1,1,0
1,262995061304852481,18,133,False,0,True,2,False,False,False,False,True,14,0.0,0.0,1,0,0,0
2,262979898002534400,17,116,False,0,False,0,False,False,False,False,False,5,0.0,0.0,0,2,1,0
3,262996108400271360,4,46,False,0,False,0,False,False,False,False,False,1,0.0,2.0,0,2,1,0
4,263018881839411200,10,90,False,0,False,0,False,False,False,False,False,1,1.0,0.0,0,4,1,0


In [16]:
df_tweets_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14483 entries, 0 to 14482
Data columns (total 19 columns):
tweet_id                     14483 non-null int64
 num_words                   14483 non-null int64
 text_length                 14483 non-null int64
 contains_questmark          14483 non-null bool
 num_questmark               14483 non-null int64
 contains_exclammark         14483 non-null bool
 num_exclammark              14483 non-null int64
 contains_happyemo           14483 non-null bool
 contains_sademo             14483 non-null bool
 contains_firstorderpron     11816 non-null object
 contains_secondorderpron    11816 non-null object
 contains_thirdorderpron     11816 non-null object
 num_uppercasechars          14483 non-null int64
 num_possentiwords           11816 non-null float64
 num_negsentiwords           11816 non-null float64
 num_mentions                14483 non-null int64
 num_hashtags                14483 non-null int64
 num_URLs                    14483 non

### Columns name cleaning

- Remove leading whitespace in the column name

In [17]:
df_tweets_features.columns = df_tweets_features.columns.str.strip()

### Imputation on orderpron features

Taking for instance, *contains_firstorderpron* column, it can be seen that the nan value is caused by the tweet text, in this case, text having all hashtags and url and thus not having necessary information to infer *contain_firstorderpron* information, so all the nan value is replaced by False

In [18]:
first_orderpron_nan = df_tweets_features.loc[:, 'contains_firstorderpron'].isna()
df_tweets_features.loc[first_orderpron_nan,:].head(3)

,tweet_id,num_words,text_length,contains_questmark,num_questmark,contains_exclammark,num_exclammark,contains_happyemo,contains_sademo,contains_firstorderpron,contains_secondorderpron,contains_thirdorderpron,num_uppercasechars,num_possentiwords,num_negsentiwords,num_mentions,num_hashtags,num_URLs,num_retweets
8,263111677485142017,5,69,False,0,False,0,False,False,NaN,NaN,NaN,0,NaN,NaN,0,5,1,0
9,262977091983785985,2,36,False,0,False,0,False,False,NaN,NaN,NaN,0,NaN,NaN,0,2,1,0
11,263129115207536640,3,45,False,0,False,0,False,False,NaN,NaN,NaN,0,NaN,NaN,0,3,1,0


In [19]:
order_pron = ['contains_firstorderpron', 'contains_secondorderpron', 'contains_thirdorderpron']
df_tweets_features.loc[:, order_pron] = df_tweets_features.loc[:, order_pron].fillna(False)

### Imputation on number sentiment word features

Similarly, the number of sentiment word features can be set to 0

In [20]:
entiwords = ['num_possentiwords', 'num_negsentiwords']
df_tweets_features.loc[:, entiwords] = df_tweets_features.loc[:, entiwords].fillna(0)

### Save to pickle

In [21]:
df_tweets_features.to_pickle('dev_tweets_features.pkl')

## Data Combining

In [22]:
df_tweets_features = pd.read_pickle('dev_tweets_features.pkl')
df_images = pd.read_pickle('dev_images.pkl')
df_raw_tweets = pd.read_pickle('dev_raw_tweets.pkl')
df_users = pd.read_pickle('dev_users.pkl')

## Merging raw tweets with tweet features
- Merge raw tweets with tweet features by the tweetId which is unique

In [23]:
df = pd.merge(df_raw_tweets, df_tweets_features, left_on = 'tweetId', right_on = 'tweet_id', how = 'inner')
df.drop('tweetId', axis = 1, inplace = True)

## Merging users with the combined dataframe
- Merge users with the dataframe by the tweet_id which is unique

In [24]:
df = pd.merge(df, df_users, on = 'tweet_id', how = 'inner')

## Merging images with the combined dataframe
- Merge images with the dataframe by imageId which is unique
- The number of rows of the resulting dataframe had been halved. This is because some images had not been found or unable to process

In [25]:
df = pd.merge(df, df_images, left_on = 'imageId(s)', right_on = 'imageId', how = 'inner')
df.drop('imageId(s)', axis = 1, inplace = True)

## Dropping irrelevant features

Features | Reason
-|-
userId | Offer no additional info
username | Offer no additional info
tweet_id | Offer no additional info

In [26]:
drop_cols = ['userId', 'username', 'tweet_id']
df.drop(drop_cols, axis = 1, inplace = True)

### Converting Timestamp to index

In [27]:
# Some of the timestamp has an extra whitespace, so we need to remove that whitespace
df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].str.replace(': ', ':')

df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'timestamp'], \
               format = '%a %b %d %H:%M:%S +0000 %Y')

df.set_index('timestamp', inplace = True)

### Converting Boolean values to int

- Get all the columns which has boolean dtype
- Apply lambda function to convert boolean to integer elementwise 

In [28]:
bool_cols = df.columns[df.dtypes == 'bool']
df.loc[:, bool_cols] = df.loc[:, bool_cols].applymap(lambda x : 1 if x else 0)

### Converting label to int
- Convert label features to integer
- There is another label called humor, which according to the paper, can be treated as fake
- If you compare with annotation column found in images.txt, the corresponding label is fake for all humor values 
- For real label, it is 1. For fake label, it is 0

In [29]:
df['label'].value_counts()

fake     6634
real     4834
humor    2604
Name: label, dtype: int64

In [30]:
df.loc[:, 'label'] = df.loc[:, 'label'].apply(lambda x : 1 if x == 'real' else 0)

### Remove events that has too little images
- Subsequent images processing require quite a large set of images to make prediction
- We will drop events that has lesser than 10 images

In [31]:
drop_image = df.loc[:, 'imageId'].drop_duplicates().str.split('_').str.get(0).value_counts() < 10
drop_idx = df.loc[:, 'imageId'].apply(lambda x: x.split('_')[0] not in drop_image.index[drop_image])
df = df.loc[drop_idx, :]

In [32]:
df.loc[:, 'event'].value_counts()

sandy                 12256
malaysia                501
boston                  404
sochi                   401
columbianChemicals      185
Name: event, dtype: int64

### Save to pickle

In [33]:
df.to_pickle('dev.pkl')

### Features

- The majority of the features had been converted to integer or float
- The rest of the features that are still objects will be used as such

Object features | Reason
-|-
tweetText | To be used for image captioning  
image_id | To be used for images statistics
image_path | To be used for image_captioning
event | To be used for EDA

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13747 entries, 2012-10-29 22:34:01 to 2014-03-19 14:43:18
Data columns (total 30 columns):
tweetText                   13747 non-null object
label                       13747 non-null int64
num_words                   13747 non-null int64
text_length                 13747 non-null int64
contains_questmark          13747 non-null int64
num_questmark               13747 non-null int64
contains_exclammark         13747 non-null int64
num_exclammark              13747 non-null int64
contains_happyemo           13747 non-null int64
contains_sademo             13747 non-null int64
contains_firstorderpron     13747 non-null int64
contains_secondorderpron    13747 non-null int64
contains_thirdorderpron     13747 non-null int64
num_uppercasechars          13747 non-null int64
num_possentiwords           13747 non-null float64
num_negsentiwords           13747 non-null float64
num_mentions                13747 non-null int64
num_hashtags      